<a href="https://colab.research.google.com/github/MMaggieZhou/sentiment_analysis/blob/main/sentiment_analysis_bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score

import spacy
from scipy.sparse import hstack
import matplotlib.pyplot as plt

In [4]:
import re
import unicodedata

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Normalize unicode characters
    text = unicodedata.normalize("NFKC", text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters (optional)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    return text

def load_data(file):
    return pd.read_csv(file, names=['id', 'entity', 'label', 'text']).set_index('id')

def clean_data(df):
    return df.drop_duplicates().dropna()

def drop_label(df):
    return df[df['label'].isin({'Positive','Negative'})]

label_encoder = LabelBinarizer()
def preprocess(df, training):
    if training:
        df['label_num'] = label_encoder.fit_transform(df['label'])
    else:
        df['label_num'] = label_encoder.transform(df['label'])
    df['text_processed'] = df['text'].apply(preprocess_text)
    return clean_data(df)

In [5]:
training_df = load_data("/content/twitter_training.csv")
training_df = drop_label(clean_data(training_df))
training_df = preprocess(training_df, training=True)
test_df = load_data("/content/twitter_validation.csv")
test_df = drop_label(clean_data(test_df))
test_df = preprocess(test_df, training=False)

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(list(texts), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
train_encodings = tokenize_texts(training_df['text_processed'], tokenizer)
val_encodings = tokenize_texts(test_df['text_processed'], tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Prepare datasets
train_dataset = TweetDataset(train_encodings, training_df['label_num'])
val_dataset = TweetDataset(val_encodings, test_df['label_num'])

# Define DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load Pre-trained BERT Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 Sentiment classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# Define Loss and Optimizer
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# Training Function
def train(model, train_loader, optimizer, loss_fn, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# Training the Model
train(model, train_loader, optimizer, loss_fn, device, epochs=3)

Epoch 1, Loss: 0.2741
Epoch 2, Loss: 0.0914
Epoch 3, Loss: 0.0522


In [18]:
def get_accuracy_score(model, loader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, axis=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return accuracy_score(true_labels, predictions)



In [19]:
test_accuracy = get_accuracy_score(model, val_loader, device)
print(f"Test Accuracy: {test_accuracy:.4f}")

train_accuracy = get_accuracy_score(model, train_loader, device)
print(f"Train Accuracy: {train_accuracy:.4f}")

Test Accuracy: 0.9853
Train Accuracy: 0.9853
